# Importiere benötigte Bibliotheken
In diesem Notebook werden die notwendigen Bibliotheken importiert.

In [29]:
import requests
import json
from pyspark.sql import SparkSession
from google.cloud import storage
from google.oauth2 import service_account

# Initialisiere Spark Session
In diesem Notebook wird die Spark Session vorbereitet.


In [30]:
# Beispiel für das Hinzufügen des GCS Connectors zu einer lokalen Spark-Session
spark = SparkSession.builder \
    .appName("MockServerToGCS") \
    .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.2") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json") \
    .getOrCreate()


24/06/16 15:09:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Abrufen der Daten vom Mockserver
In diesem Notebook werden die Daten von einem Mockserver abgerufen.


In [31]:
# Mockserver URL
mockserver_url = 'http://localhost:3000/applicants'

# Daten von Mockserver abrufen
response = requests.get(mockserver_url)
if response.status_code != 200:
    raise Exception(f"Failed to retrieve data: {response.status_code}")

try:
    data = response.json()
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Response text:", response.text)
    raise


# Speichern der Daten in eine temporäre JSON-Datei
In diesem Notebook werden die abgerufenen Daten in eine temporäre JSON-Datei gespeichert.


In [32]:
# Temporärer Pfad für die JSON-Datei
temp_json_path = '/tmp/mock_data.json'

# Speichern der Daten in eine JSON-Datei
with open(temp_json_path, 'w') as f:
    json.dump(data, f)


# Laden der JSON-Datei in einen Spark DataFrame
In diesem Notebook werden die Daten aus der JSON-Datei in einen Spark DataFrame geladen.


In [33]:
# JSON-Datei in einen Spark DataFrame laden
df = spark.read.json(temp_json_path)

# Optional: DataFrame anzeigen
df.show()


+--------------------+--------------------+---------------+------------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+----------------+--------------------+--------------------+--------------------+--------------------+------------+--------+------------+--------------------+--------------------+----------------------+--------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|  Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|    Nachname|Standort|      Status|           StellenID|       Telefonnummer|Veröffentlichungsdatum| Vorname|  id|
+--------------------+--------------------+---------------+---------

# Konfiguration für Google Cloud Storage
In diesem Notebook wird die Google Cloud konfiguriert.


In [36]:
# Namen des Google Cloud Storage Buckets und der Zieldatei festlegen
bucket_name = 'prod_prototype'
destination_blob_name = 'bronze/applicant_data_raw'

# Dienstkonto-Datei laden
service_account_json = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json'
credentials = service_account.Credentials.from_service_account_file(service_account_json)

# Google Cloud Storage Client initialisieren
client = storage.Client(credentials=credentials, project='prototyp-etl-pipline')
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)


# Speichern des DataFrames als JSON auf Google Cloud Storage
In diesem Notebook wird der DataFrame als JSON-Datei auf Google Cloud Storage gespeichert.


In [38]:
# DataFrame als JSON auf Google Cloud Storage speichern
output_path = 'gs://{}/{}'.format(bucket_name, destination_blob_name)
df.write.mode('overwrite').json(output_path)

# Optional: Überprüfen, ob die Datei erfolgreich hochgeladen wurde
blobs = list(bucket.list_blobs(prefix=destination_blob_name))
if len(blobs) > 0:
    print(f"File successfully uploaded to {destination_blob_name} in bucket {bucket_name}.")
else:
    print("File upload failed.")


Py4JJavaError: An error occurred while calling o67.json.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:454)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:530)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:388)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:361)
	at org.ap

# Beenden der Spark Session
In diesem Notebook wird die Spark Session beendet.


In [ ]:
# Spark Session beenden
spark.stop()
